In [ ]:
#@title Load libraries

import os
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import keras.backend as K
from tensorflow import keras


from json import loads
from cleantext import clean
from hazm import Normalizer, Lemmatizer, word_tokenize
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Embedding, Dropout, BatchNormalization, Flatten
from tensorflow.keras.layers import GlobalMaxPool1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, LSTM, GRU, Bidirectional, SimpleRNN
from tensorflow.keras.layers import multiply, Input, Concatenate
from tensorflow.keras.optimizers import Adam, schedules
from tensorflow.keras.regularizers import l2
from tensorflow.keras import losses, metrics

In [6]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text):
    text = text.strip()

    # regular cleaning
    text = clean(text,
      fix_unicode=True,
      to_ascii=False,
      lower=True,
      no_line_breaks=True,
      no_urls=True,
      no_emails=True,
      no_phone_numbers=True,
      no_numbers=False,
      no_digits=False,
      no_currency_symbols=True,
      no_punct=False,
      replace_with_url="",
      replace_with_email="",
      replace_with_phone_number="",
      replace_with_number="",
      replace_with_digit="0",
      replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)

    # normalizing
    normalizer = Normalizer()
    text = normalizer.normalize(text)

    # removing wierd patterns
    wierd_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      u"\U0001f926-\U0001f937"
      u'\U00010000-\U0010ffff'
      u"\u200d"
      u"\u2640-\u2642"
      u"\u2600-\u2B55"
      u"\u23cf"
      u"\u23e9"
      u"\u231a"
      u"\u3030"
      u"\ufe0f"
      u"\u2069"
      u"\u2066"
      # u"\u200c"
      u"\u2068"
      u"\u2067"
      "]+", flags=re.UNICODE)

    text = wierd_pattern.sub(r'', text)

    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)

    return text

In [7]:
test_dir = 'nicknames.csv'
train_dir = 'clean_data_balanced.csv'

df_train = pd.read_csv(train_dir)
df_test = pd.read_csv(test_dir)

In [8]:
df_test = df_test.rename(columns={"nick_name":"text"})

In [9]:
df_train['text'] = df_train['text'].astype(str)
df_train['text'] = df_train['text'].apply(cleaning)

In [10]:
samples = df_train['text'].tolist()
labels = df_train['is_offensive'].tolist()

In [11]:
validation_split = 0.1
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [12]:
from tensorflow.keras.layers import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=300)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(64)
vectorizer.adapt(text_ds)

In [13]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'you']

In [14]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 2149, 6396,   15,    2,    1])

In [15]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [16]:
fastTextDir = '.'
fastText_fa_path = os.path.join(fastTextDir, 'cc.fa.300.vec')
fastText_en_path = os.path.join(fastTextDir, 'cc.en.300.vec')

In [17]:
embeddings_index = {}

with open(fastText_fa_path) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

with open(fastText_en_path) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 3745941 word vectors.


In [18]:
num_tokens = len(voc) + 2
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 17028 words (2972 misses)


In [19]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)


In [20]:
from tensorflow.keras import layers

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(2, activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 300)         6000600   
                                                                 
 conv1d (Conv1D)             (None, None, 128)         192128    
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 128)        0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, None, 128)         82048     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, None, 128)        0         
 1D)                                                         

In [21]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)


In [22]:
x_train.shape

(72251, 300)

In [23]:
y_train.shape

(72251,)

In [24]:
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=(x_val, y_val))

Epoch 1/50
1129/1129 [==============================] - 10s 6ms/step - loss: 0.2230 - acc: 0.9113 - val_loss: 0.9394 - val_acc: 0.5764
Epoch 2/50
1129/1129 [==============================] - 6s 6ms/step - loss: 0.1686 - acc: 0.9342 - val_loss: 0.8799 - val_acc: 0.5769
Epoch 3/50
1129/1129 [==============================] - 6s 6ms/step - loss: 0.1503 - acc: 0.9416 - val_loss: 0.8855 - val_acc: 0.5754
Epoch 4/50
1129/1129 [==============================] - 6s 6ms/step - loss: 0.1337 - acc: 0.9487 - val_loss: 0.9789 - val_acc: 0.5773
Epoch 5/50
1129/1129 [==============================] - 6s 6ms/step - loss: 0.1158 - acc: 0.9560 - val_loss: 1.0050 - val_acc: 0.5782
Epoch 6/50
1129/1129 [==============================] - 6s 6ms/step - loss: 0.0981 - acc: 0.9637 - val_loss: 1.0866 - val_acc: 0.5784
Epoch 7/50
1129/1129 [==============================] - 6s 6ms/step - loss: 0.0815 - acc: 0.9711 - val_loss: 1.0279 - val_acc: 0.5776
Epoch 8/50
1129/1129 [==============================] - 6s 6m

In [ ]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

probabilities = end_to_end_model.predict(
    [["کیرم دهنت"]]
)

np.argmax(probabilities[0])

In [ ]:
df_train['text'] = df_train['text'].astype(str)
df_test['text'] = df_test['text'].astype(str)

df_train['text'] = df_train['text'].apply(cleaning)
df_test['text'] = df_test['text'].apply(cleaning)

train_data, val_data = train_test_split(df_train, test_size=0.2)

In [ ]:
def get_dict(df):
    wordDict = {}
    for idx, row in enumerate(df.text):
        row = re.split(r'([a-zA-Z]+)', row)
        row = " ".join(str(item) for item in row)
        words = row.split()
        for wrd in words:
            if wrd in wordDict:
                wordDict[wrd] += 1
            else:
                wordDict[wrd] = 1
    return wordDict

train_wordDict = get_dict(df_train)
test_wordDict = get_dict(df_test)

In [ ]:
fastTextDir = '.'
fastText_fa_path = os.path.join(fastTextDir, 'cc.fa.300.vec')
fastText_en_path = os.path.join(fastTextDir, 'cc.en.300.vec')

def get_embedding(wordDict):
    embeddings_index = {}
    with open(fastText_fa_path, encoding='utf8') as infile:
        for line in infile:
            values = line.split()
            word = values[0]
            try:
                coefs = np.asarray(values[1:], dtype='float32')
            except:
                print("Warnning"+str(values)+" in" + str(line))
            if word in wordDict:
                embeddings_index[word] = coefs

    with open(fastText_en_path, encoding='utf8') as infile:
        for line in infile:
            values = line.split()
            word = values[0]
            try:
                coefs = np.asarray(values[1:], dtype='float32')
            except:
                print("Warnning"+str(values)+" in" + str(line))
            if word in wordDict:
                embeddings_index[word] = coefs
    return embeddings_index


In [ ]:
train_embeddings = get_embedding(train_wordDict)
test_embeddings = get_embedding(test_wordDict)

In [ ]:
MAX_NB_WORDS = 5500
MAX_SEQUENCE_LENGTH = 350

content_train = df_train['text']
content_test = df_test['text']

y_train = np.array(df_train['is_offensive'])

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(content_train)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(content_train)
test_sequences = tokenizer.texts_to_sequences(content_test)

train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
def prepare_test(sentence):
    test_sequences = tokenizer.texts_to_sequences([sentence])
    test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
    return test_data

In [ ]:
EMBEDDING_DIM = 300
embeddings_index = train_embeddings
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [ ]:
nClasses = 2

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH, ))
embedded_sequences = embedding_layer(sequence_input)

x = BatchNormalization()(embedded_sequences)
x = Conv1D(256, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
preds = Dense(nClasses, activation='softmax')(x)
model = Model(sequence_input, preds)

optimizer = Adam(learning_rate=5e-3, beta_1=0.9, beta_2=0.999, 
                epsilon=1e-07, amsgrad=False)

model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer='rmsprop',
                metrics=metrics.SparseCategoricalAccuracy('accuracy'))

In [ ]:
BATCH_SIZE = 64
EPOCHS =   20 


model.fit(train_data, y_train.astype(float),
        validation_split=0.2, 
        epochs=EPOCHS,
        batch_size=BATCH_SIZE)
model.save(('nickname.h5'))